In [320]:
import os
from datetime import timedelta, date

import osmapi as osm
from shapely.geometry import Polygon, Point
from random import uniform
from sklearn.neighbors import DistanceMetric
from sklearn.linear_model import LinearRegression
import folium
from folium import plugins

# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Тарелкин Александр Алексеевич

Группа: MADE-ML-11

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

#### Полигон Афганистана

In [15]:
AF_RELATION_ID = 303427

api = osm.OsmApi()

relation = api.RelationGet(AF_RELATION_ID)

ways_ids = [x['ref'] for x in relation['member'] if x['type'] == 'way'] # получаем id путей границы
ways = api.WaysGet(ways_ids) # получаем сами пути

nodes_ids = [y for x in ways.values() for y in x['nd']] # получаем id точек границы
nodes_ids = [x for i,x in enumerate(nodes_ids) if i % 5 == 0] # берем каждую пятую точку границы, для быстродействия 

In [94]:
coords = {}
for i, x in enumerate(nodes_ids):
    if i % 25 == 0:
        print(i)
    node = api.NodeGet(x)
    coords[i] = (node['lat'], node['lon'])
    
af_poly = Polygon(c).convex_hull # полигон Афганистана

0
25
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
50
75
100
125
150
175
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1100
1125
1150
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2050
('Connection aborted.', RemoteDisconnected('Remote end closed connection

#### Генерация случайных точек. Наивный метод.

In [387]:
# наивный метод

up_boundary_lat = 90
down_boundary_lat = -90
up_boundary_lon = 180
down_boundary_lon = 0

m = folium.Map(
    location=[af_poly.centroid.x, af_poly.centroid.y],
    zoom_start=1,
)

p = Point((uniform(down_boundary_lat, up_boundary_lat), uniform(down_boundary_lon, up_boundary_lon)))  
i = 0
while not af_poly.contains(p):
    p = Point((uniform(down_boundary_lat, up_boundary_lat), uniform(down_boundary_lon, up_boundary_lon)))
    i += 1
    folium.Marker(
        location=[p.x, p.y],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
print(i, 'iteration')

84 iteration


In [388]:
m # визуализация - карта, с отметками сгенерированных точек

#### Генерация точек. Не наивный метод.

In [139]:
def calc_haversine_dist_from_point_to_Af(lat, lon):
    """ возвращает расстояние от точки до центра Афганистана в км """
    
    af_center_coords = list(map(lambda x: x*3.1415925/180, [af_poly.centroid.x, af_poly.centroid.y]))
    point_coords = list(map(lambda x: x*3.1415925/180, [lat, lon]))
    
    haversine = DistanceMetric.get_metric('haversine')
    
    return 6371 * haversine.pairwise([af_center_coords, point_coords])[0][1]

In [394]:
# не наивный алгоритм - иттеративно уменьшаем область, опираясь на расстояние от точки до центра полигона

x_min, x_max, y_min, y_max = down_boundary_lat, up_boundary_lat, down_boundary_lon, up_boundary_lon

p = Point((uniform(x_min, x_max), uniform(y_min, y_max))) 
coords = [[p.x, p.y]]
i = 1

while not af_poly.contains(p):
    i += 1
    corr = calc_haversine_dist_from_point_to_Af(p.x, p.y) * 180 / (6371 * 3.14159265) # расстояние в градусах до центра полигона
    
    # рассчитываем новые гранцы области
    
    x_min = max(p.x - 1.5 * corr, x_min)
    x_max = min(p.x + 1.5 * corr, x_max)
    y_min = max(p.y - 1.5 * corr, y_min)
    y_max = min(p.y + 1.5 * corr, y_max)
    
    print('x_min = ', x_min,' x_max = ', x_max, ' y_min = ', y_min, ' y_max = ', y_max)
    
    p = Point((uniform(x_min, x_max), uniform(y_min, y_max)))
    
    coords.append([p.y, p.x]) # массив координат точек, для последующей визуализации
    
    if i > 20:
        x_min, x_max, y_min, y_max = down_boundary_lat, up_boundary_lat, down_boundary_lon, up_boundary_lon

print(i, 'iteration') 

x_min =  -90  x_max =  90  y_min =  0  y_max =  110.71856619227223
x_min =  -4.566548215769629  x_max =  69.12638520073504  y_min =  59.184497913449164  y_max =  110.71856619227223
x_min =  18.93363089205402  x_max =  38.50450137714261  y_min =  61.0781075893311  y_max =  80.64897807441969
x_min =  18.93363089205402  x_max =  38.50450137714261  y_min =  61.0781075893311  y_max =  80.64897807441969
x_min =  18.93363089205402  x_max =  38.50450137714261  y_min =  61.0781075893311  y_max =  80.64897807441969
x_min =  18.93363089205402  x_max =  38.14374940223924  y_min =  61.0781075893311  y_max =  73.2639297849718
x_min =  18.93363089205402  x_max =  38.14374940223924  y_min =  61.0781075893311  y_max =  73.2639297849718
8 iteration


In [393]:
# Анимированная визуализация

start_date = datetime.today()

times = [single_date.strftime('%Y-%m-%d') for single_date in (start_date + timedelta(n) for n in range(len(coords)))]

points = [
    {
        'time': time,
        'coordinates': point
    } for time, point in zip(times, coords)
]

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'id': 'point',
            'icon': 'marker',
            'iconstyle': {
                'iconUrl': 'https://leafletjs.com/examples/geojson/baseball-marker.png',
                'iconSize': [20, 20]
            }
        }
    } for point in points
]

features.append(
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coords,
        },
        'properties': {
            'popup': 'Current address',
            'times': times,
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'green',
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': 2
            },
            'style': {'weight': 0},
            'id': 'man'
        }
    }
)

m = folium.Map(
    location=[af_poly.centroid.x, af_poly.centroid.y],
    tiles='cartodbpositron',
    zoom_start=1,
)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY/MM/DD',
    time_slider_drag_update=True,
    duration='P13D'
).add_to(m)

m.save(os.path.join('Plugins_1.html'))
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here